In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Load the CSV file
df = pd.read_csv('../../data/01_raw_data/03_retail_trade/retail_trade.csv')

In [3]:
# Check the NAICS column specifically
print("\n=== NAICS COLUMN ANALYSIS ===")
naics_col = 'North American Industry Classification System (NAICS)'

# Count unique values
print(f"Unique NAICS categories: {df[naics_col].nunique()}")

# Check for any missing/null values
print(f"Missing values in NAICS: {df[naics_col].isnull().sum()}")

# Display all unique NAICS values to see the full list
print(f"\nAll unique NAICS categories:")
print("="*50)
for i, category in enumerate(df[naics_col].unique(), 1):
    print(f"{i:2d}. {category}")


=== NAICS COLUMN ANALYSIS ===
Unique NAICS categories: 30
Missing values in NAICS: 0

All unique NAICS categories:
 1. Retail trade [44-45]
 2. Motor vehicle and parts dealers [441]
 3. Automobile dealers [4411]
 4. New car dealers [44111]
 5. Used car dealers [44112]
 6. Other motor vehicle dealers [4412]
 7. Automotive parts, accessories and tire retailers [4413]
 8. Building material and garden equipment and supplies dealers [444]
 9. Food and beverage retailers [445]
10. Grocery and convenience retailers [4451]
11. Supermarkets and other grocery retailers (except convenience retailers) [44511]
12. Convenience retailers and vending machine operators [44513]
13. Specialty food retailers [4452]
14. Beer, wine and liquor retailers [4453]
15. Furniture, home furnishings, electronics and appliances retailers [449]
16. Furniture, floor covering, window treatment and other home furnishings retailers [4491]
17. Furniture retailers [44911]
18. Floor covering, window treatment and other home

In [4]:
# Filter for cannabis retailers
cannabis_df = df[df['North American Industry Classification System (NAICS)'].str.contains('Cannabis retailers', na=False)]

# Display the filtered data
print("Cannabis retail sales data:")
print(f"Found {len(cannabis_df)} records")
cannabis_df.head()

Cannabis retail sales data:
Found 1974 records


,REF_DATE,GEO,DGUID,North American Industry Classification System (NAICS),Sales,Adjustments,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
11400,2018-10,Canada,2021A000011124,Cannabis retailers [459993],Total retail sales,Unadjusted,Dollars,81,thousands,3,v1446859541,1.30.1.1,42084.0,NaN,NaN,NaN,0
11401,2018-10,Canada,2021A000011124,Cannabis retailers [459993],Total retail sales,Seasonally adjusted,Dollars,81,thousands,3,v1446870213,1.30.1.2,42084.0,NaN,NaN,NaN,0
11432,2018-10,Newfoundland and Labrador,2021A000210,Cannabis retailers [459993],Total retail sales,Unadjusted,Dollars,81,thousands,3,v1446859572,2.30.1.1,2144.0,NaN,NaN,NaN,0
11463,2018-10,Prince Edward Island,2021A000211,Cannabis retailers [459993],Total retail sales,Unadjusted,Dollars,81,thousands,3,v1446859603,3.30.1.1,850.0,NaN,NaN,NaN,0
11494,2018-10,Nova Scotia,2021A000212,Cannabis retailers [459993],Total retail sales,Unadjusted,Dollars,81,thousands,3,v1446859634,4.30.1.1,4272.0,NaN,NaN,NaN,0


In [5]:
print("Column names:")
print(df.columns.tolist())
print(f"\nTotal records: {len(df)}")
print(f"Cannabis records: {len(cannabis_df)}")

Column names:
['REF_DATE', 'GEO', 'DGUID', 'North American Industry Classification System (NAICS)', 'Sales', 'Adjustments', 'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'VALUE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']

Total records: 72054
Cannabis records: 1974


In [6]:
# Load the cannabis dataset
#cannabis_df = pd.read_csv('cannabis_retail_sales.csv')

print("=== INITIAL DATA INSPECTION ===")
print(f"Dataset shape: {cannabis_df.shape}")
print(f"Memory usage: {cannabis_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("\nColumn names and types:")
print(cannabis_df.dtypes)

=== INITIAL DATA INSPECTION ===
Dataset shape: (1974, 17)
Memory usage: 1.34 MB

Column names and types:
REF_DATE                                                  object
GEO                                                       object
DGUID                                                     object
North American Industry Classification System (NAICS)     object
Sales                                                     object
Adjustments                                               object
UOM                                                       object
UOM_ID                                                     int64
SCALAR_FACTOR                                             object
SCALAR_ID                                                  int64
VECTOR                                                    object
COORDINATE                                                object
VALUE                                                    float64
STATUS                                            

In [7]:
print("\n=== DATA QUALITY ASSESSMENT ===")

# Check for duplicates
duplicates = cannabis_df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

# Missing values analysis
print("\nMissing values by column:")
missing_data = cannabis_df.isnull().sum()
missing_percent = (missing_data / len(cannabis_df)) * 100
missing_summary = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percentage': missing_percent
}).sort_values('Missing_Count', ascending=False)
print(missing_summary[missing_summary['Missing_Count'] > 0])

# Check data types and unique values for key columns
print(f"\nUnique values in key columns:")
key_columns = ['REF_DATE', 'GEO', 'Sales', 'Adjustments', 'STATUS', 'SYMBOL']
for col in key_columns:
    if col in cannabis_df.columns:
        print(f"{col}: {cannabis_df[col].nunique()} unique values")





=== DATA QUALITY ASSESSMENT ===
Duplicate rows: 0

Missing values by column:
            Missing_Count  Missing_Percentage
SYMBOL               1974          100.000000
TERMINATED           1974          100.000000
STATUS               1075           54.457953
VALUE                 173            8.763931

Unique values in key columns:
REF_DATE: 83 unique values
GEO: 23 unique values
Sales: 1 unique values
Adjustments: 2 unique values
STATUS: 4 unique values
SYMBOL: 0 unique values


In [8]:
import pandas as pd
import numpy as np

def clean_cannabis_data(cannabis_df):
    print("=== STRICT DATA CLEANING: 50% MISSING VALUE THRESHOLD ===")
    print(f"Original shape: {cannabis_df.shape}")

    # Step 1: Check data types
    print("\n1. DATA TYPES ANALYSIS:")
    print(cannabis_df.dtypes)

    # Step 2: Check missing values by column
    print("\n2. MISSING VALUES BY COLUMN:")
    missing_analysis = pd.DataFrame({
        'Column': cannabis_df.columns,
        'Missing_Count': cannabis_df.isnull().sum(),
        'Missing_Percentage': (cannabis_df.isnull().sum() / len(cannabis_df)) * 100,
        'Data_Type': cannabis_df.dtypes
    })
   # print(missing_analysis[['Column', 'Missing_Percentage']])
    
    # Step 3: Drop columns with 50%+ missing data
    print("\n3. DROPPING COLUMNS WITH 50%+ MISSING DATA:")
    columns_to_drop = missing_analysis[missing_analysis['Missing_Percentage'] >= 50]['Column'].tolist()

    if columns_to_drop:
        print(f"Columns to drop: {columns_to_drop}")
        df_clean = cannabis_df.drop(columns=columns_to_drop)
        print(f"Dropped {len(columns_to_drop)} columns")
    else:
        print("No columns have 50%+ missing data")
        df_clean = cannabis_df.copy()

    print(f"Shape after column removal: {df_clean.shape}")

    # Step 4: Check missing values by row
    print("\n4. MISSING VALUES BY ROW:")
    row_missing_counts = df_clean.isnull().sum(axis=1)
    row_missing_percentage = (row_missing_counts / df_clean.shape[1]) * 100

    print(f"Rows with 50%+ missing data: {(row_missing_percentage >= 50).sum()}")
    print(f"Total rows: {len(df_clean)}")

    # Step 5: Drop rows with 50%+ missing data
    print("\n5. DROPPING ROWS WITH 50%+ MISSING DATA:")
    rows_to_keep = row_missing_percentage < 50
    df_clean = df_clean[rows_to_keep]

    rows_dropped = len(cannabis_df) - len(df_clean)
    print(f"Rows dropped: {rows_dropped}")
    print(f"Shape after row removal: {df_clean.shape}")

    # New Step: Drop rows having missing values in the "value" column
    print( "\n6. DROPPING ROWS WITH MISSING data in 'value':" )
    initial_row_count = len(df_clean)
    df_clean = df_clean.dropna(subset=['VALUE'])
    rows_dropped_value_col = initial_row_count - len(df_clean)
    print(f"Rows dropped due to missing data in 'value': {rows_dropped_value_col}")
    print(f"Shape after 'value' column removal: {df_clean.shape}")

    # Step 6: Final data type optimization
    print("\n6. OPTIMIZING DATA TYPES:")

    # Convert REF_DATE to datetime if it's object
    if 'REF_DATE' in df_clean.columns and df_clean['REF_DATE'].dtype == 'object':
        df_clean['REF_DATE'] = pd.to_datetime(df_clean['REF_DATE'])
        print("   ✓ REF_DATE converted to datetime")

    # Convert VALUE to numeric if it's object
    if 'VALUE' in df_clean.columns and df_clean['VALUE'].dtype == 'object':
        df_clean['VALUE'] = pd.to_numeric(df_clean['VALUE'], errors='coerce')
        print("   ✓ VALUE converted to numeric")

    # Convert text columns to category to save memory
    text_columns = ['GEO', 'Sales', 'Adjustments', 'North American Industry Classification System (NAICS)']
    for col in text_columns:
        if col in df_clean.columns and df_clean[col].dtype == 'object':
            df_clean[col] = df_clean[col].astype('category')
            print(f"   ✓ {col} converted to category")

    # Step 7: Final summary
    print(f"\n7. FINAL CLEANING SUMMARY:")
    print(f"Original shape: {cannabis_df.shape}")
    print(f"Final shape: {df_clean.shape}")
    print(f"Columns dropped: {len(columns_to_drop)}")
    print(f"Rows dropped: {rows_dropped + rows_dropped_value_col}")
    print(f"Data retention: {(len(df_clean)/len(cannabis_df))*100:.1f}%")

    # Check remaining missing values
    print(f"\n8. REMAINING MISSING VALUES:")
    remaining_missing = df_clean.isnull().sum()
    if remaining_missing.sum() > 0:
        print(remaining_missing[remaining_missing > 0])
    else:
        print("No missing values remaining")

    print(f"\n9. FINAL DATA TYPES:")
    print(df_clean.dtypes)

    return df_clean

# Apply strict cleaning
cannabis_retail_clean = clean_cannabis_data(cannabis_df)


=== STRICT DATA CLEANING: 50% MISSING VALUE THRESHOLD ===
Original shape: (1974, 17)

1. DATA TYPES ANALYSIS:
REF_DATE                                                  object
GEO                                                       object
DGUID                                                     object
North American Industry Classification System (NAICS)     object
Sales                                                     object
Adjustments                                               object
UOM                                                       object
UOM_ID                                                     int64
SCALAR_FACTOR                                             object
SCALAR_ID                                                  int64
VECTOR                                                    object
COORDINATE                                                object
VALUE                                                    float64
STATUS                                       

In [9]:
print("\n=== EDA: DATASET OVERVIEW ===")

# Time range
print(f"Date range: {cannabis_retail_clean['REF_DATE'].min()} to {cannabis_retail_clean['REF_DATE'].max()}")
print(f"Total months covered: {cannabis_retail_clean['REF_DATE'].nunique()}")

# Geographic coverage
print(f"\nGeographic areas covered:")
geo_counts = cannabis_retail_clean['GEO'].value_counts()
print(geo_counts)

# Sales types
print(f"\nSales types:")
sales_counts = cannabis_retail_clean['Sales'].value_counts()
print(sales_counts)

# Adjustment types
print(f"\nAdjustment types:")
adj_counts = cannabis_retail_clean['Adjustments'].value_counts()
print(adj_counts)

# values
print(f"\nsales values (in millions $):")
val_col = cannabis_retail_clean['VALUE']
sales_in_m = val_col/1000
stats = sales_in_m.describe()
print(stats)


=== EDA: DATASET OVERVIEW ===
Date range: 2018-10-01 00:00:00 to 2025-08-01 00:00:00
Total months covered: 83

Geographic areas covered:
GEO
Canada                         166
Alberta                         83
British Columbia                83
Newfoundland and Labrador       83
New Brunswick                   83
Montréal, Quebec                83
Ontario                         83
Toronto, Ontario                83
Saskatchewan                    83
Nova Scotia                     83
Quebec                          83
Manitoba                        81
Calgary, Alberta                80
Edmonton, Alberta               80
Winnipeg, Manitoba              80
Quebec, Quebec                  80
Ottawa, Ontario                 77
Gatineau, Quebec                76
Vancouver, British Columbia     75
Prince Edward Island            74
Yukon                           72
Northwest Territories           30
Name: count, dtype: int64

Sales types:
Sales
Total retail sales    1801
Name: count, dt

In [10]:
# Save the filtered data to a new CSV file
cannabis_retail_clean.to_csv('../../data/02_processed_data/cannabis_retail_sales.csv', index=False)